# Hotdog classification: Train SVM on top of VGG19 FC layer output*

*Classifier of VGG19 updated to produce non-class-score output result

In [50]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.autograd import Variable
import torchvision
import numpy as np
import os
import time
import copy
import pandas as pd

In [2]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [31]:
import sys
if sys.platform == 'win32':
    data_dir = 'E:\GuestUbuntuShared\hotdogs_dataset_train'
else:
    data_dir = os.path.join(os.environ['HOME'], 'hse/data/hotdogs_dataset_train')
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=1,
                                              shuffle=True, num_workers=4)
              for x in ['train']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train']}
class_names = image_datasets['train'].classes
print(class_names)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

['hotdog', 'not_hotdog']


In [19]:
print(models.vgg19())

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): ReLU(inplace)

### Update forward pass of VGG19

In [44]:
for param in cnn.parameters():
    param.requires_grad = False
    
def clear_last_layer(model):
    del model.classifier[6]
    del model.classifier[5]
    del model.classifier[4]
    return model

cnn = clear_last_layer(models.vgg19(pretrained=True))
    
def custom_forward(x):
    x = cnn.features(x)
    x = x.view(x.size(0), -1)
    x = cnn.classifier(x)
    return x.cpu().data.numpy()[0]

cnn.forward = custom_forward

In [45]:
cnn.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace)
  (2): Dropout(p=0.5)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
)

In [54]:
# License: https://github.com/alexanderkuk/log-progress/blob/master/LICENSE
def log_progress(sequence, every=None, size=None, name='Items'):
    """Progress bar for Jupyter Notebook"""
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index)
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size)
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?'))

In [48]:
cnn = cnn.to(device)
train_data = []
train_labels = []
for inputs, labels in log_progress(dataloaders['train'], every=5):
    inputs = inputs.to(device)
    train_data.append(cnn(inputs))
    train_labels.append(labels.data.numpy()[0])

In [55]:
train_df = pd.DataFrame(train_data)
labels_df = pd.Series(train_labels)

In [56]:
train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,-6.732235,-0.115475,-7.367240,-6.015232,-1.609133,-6.970767,-5.815784,3.400264,-8.163808,-4.497988,...,8.764155,-0.957648,-7.635461,-2.058371,-9.433183,-1.278329,2.609164,-1.215327,-5.124540,0.773738
1,-2.476744,-2.350722,-2.985142,-4.069021,-4.841514,-4.735853,-3.610507,0.274117,-6.328979,-3.687347,...,4.128976,0.733067,-5.086288,-5.086081,-4.106370,-1.170274,0.990233,-0.360534,-5.015172,1.644582
2,1.222328,-1.861529,0.373557,-6.381817,-4.266272,-4.156070,-1.971125,-2.656559,-2.522385,-1.621938,...,-2.502311,0.236949,-1.876375,-2.796245,-0.234787,-0.097991,1.026815,-4.200680,-1.127133,-0.279727
3,-1.120292,-0.335988,-2.142336,-1.229021,-1.567264,-0.797783,-2.187281,-1.618185,-2.833913,-2.300833,...,0.572388,1.747288,-3.676460,-1.040054,0.336179,0.050301,-1.865850,0.171282,-1.450607,-2.581167
4,-3.485689,-0.817867,-3.114861,-3.242221,-0.665309,-3.256441,-1.970002,2.749093,-3.946038,-0.904642,...,1.488968,0.723232,-2.759034,-1.401602,-3.439389,-1.088350,-2.176195,-0.520471,-0.882599,-1.668488


In [60]:
labels_df.head()

0    0
1    0
2    1
3    1
4    0
dtype: int64

## Classifier - SVC

In [101]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.svm import SVC
import scipy as sp

def random_search_cv():
    return RandomizedSearchCV(
        SVC(),
        param_distributions = {
            'C': sp.stats.uniform(loc=0.0001, scale=5.0),
            'kernel': ['rbf', 'linear', 'poly', 'sigmoid'],
        },
        cv=None
    )
def grid_search_cv():
    return GridSearchCV(
        SVC(),
        param_grid = {
            'C': np.arange(0.1, 1.0, 0.1),
            'kernel': ['rbf', 'linear'],
        },
        cv=None
    )

In [59]:
search = random_search_cv()

In [61]:
search.fit(train_df, labels_df)

RandomizedSearchCV(cv=None, error_score='raise',
          estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
          fit_params=None, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000018B3FF223C8>, 'kernel': ['rbf', 'linear', 'poly', 'sigmoid']},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

In [78]:
best_cv_err = 1 - search.best_score_
print(best_cv_err)
print(search.best_score_)
print(search.best_params_)

0.02333931777378817
0.9766606822262118
{'C': 1.6696255302434393, 'kernel': 'linear'}


In [85]:
detailed_search = grid_search_cv()
detailed_search.fit(train_df, labels_df)

GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]), 'kernel': ['rbf', 'linear']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [86]:
best_cv_err = 1 - detailed_search.best_score_
print(best_cv_err)
print(detailed_search.best_score_)
print(detailed_search.best_params_)

0.02333931777378817
0.9766606822262118
{'C': 0.1, 'kernel': 'linear'}
